# Lispy

Veja o tutorial do Peter Norvig em http://norvig.com/lispy.html. Vamos implementar
um interpretador de Lisp/Scheme em Python.

In [75]:
# Imports
from lark import Lark, InlineTransformer
import math
import operator as op
from collections import deque, ChainMap


# Apelidos
class Symbol:
    def __init__(self, st):
        self.data = st
        
    def __repr__(self):
        return self.data
    
    def __hash__(self):
        return hash(self.data)
    
    def __eq__(self, other):
        if isinstance(other, Symbol):
            return self.data == other.data
        return NotImplemented
    
Ast = list      # árvore sintática
Ctx = ChainMap  # contexto de execução

# Análise sintática + léxica

A principal função é eval_scheme(), que analisa uma string de código e retorna a árvore sintática correspondente.

In [153]:
# Contexto de execução
default_context = {
    Symbol('+'): op.add,
    Symbol('-'): op.sub,
    Symbol('*'): op.mul,
    Symbol('/'): op.truediv,
    Symbol('<'): op.lt,
    Symbol('<='): op.le,
    Symbol('sqrt'): math.sqrt,
}

grammar = r'''
start : value*

?value : INT     -> int
      | FLOAT   -> float
      | STRING  -> string
      | SYMBOL  -> symbol
      | "nil"   -> nil
      | bool
      | quoted
      | sexpr
      | infix

bool : "#t" -> true | "true" -> true | "🙂" -> true
     | "#f" -> false | "false" -> false | "🙁" -> false

infix: "[" value value value "]" 

quoted : "'" value

sexpr : "(" value* ")"

FLOAT.2  : /\d+\.\d+/
INT.1    : /\d+/
SYMBOL.0 : /[-\w+*\/!$@<>]+/
STRING   : /"[^"]*"/

%ignore /;[^\n]*/
%ignore /\s+/
'''

class LispyPPTransformer(InlineTransformer):
    int = int
    float = float
    
    def start(self, *args):
        if len(args) == 1:
            return args[0]
        return [Symbol('begin'), *args]
    
    def string(self, x):
        return x[1:-1]
    
    def true(self):
        return True
    
    def false(self):
        return False
    
    def nil(self):
        return None
    
    def symbol(self, x):
        return Symbol(x)
    
    def quoted(self, x):
        return [Symbol('quote'), x]
    
    def sexpr(self, *args):
        return [*args]
    
    def infix(self, x, y, z):
        if(y == Symbol("<-")):
            return [Symbol("define"), x, z]
        else:
            return [y, x, z]
    
    
lark = Lark(grammar, parser='lalr', transformer=LispyPPTransformer())


def eval_scheme(st: str, ctx=default_context):
    """
    Avalia a string de código Scheme no contexto padrão.
    """
    
    ast = parse(st)
    return run_ast(ast, ChainMap({}, ctx))


def parse(st: str) -> Ast:
    """
    Realiza análise sintática da string de código e retorna uma 
    árvore sintática
    """
    ast = lark.parse(st)
    #for tk in lark.lex(st): print(repr(tk))
#     print(ast.pretty())
    return ast

In [136]:
st = """
    (define x 42)
    (define outra-coisa)
    ; Comentario
    [4 + 5]
"""

parse(st)

Token(LPAR, '(')
Token(SYMBOL, 'define')
Token(SYMBOL, 'x')
Token(INT, '42')
Token(RPAR, ')')
Token(LPAR, '(')
Token(SYMBOL, 'define')
Token(SYMBOL, 'outra-coisa')
Token(RPAR, ')')
Token(LSQB, '[')
Token(INT, '4')
Token(SYMBOL, '+')
Token(INT, '5')
Token(RSQB, ']')


[begin, [define, x, 42], [define, outra-coisa], [+, 4, 5]]

In [77]:
st = '(+ 40 "2")'
parse(st)

Token(LPAR, '(')
Token(SYMBOL, '+')
Token(INT, '40')
Token(STRING, '"2"')
Token(RPAR, ')')
start
  value
    sexpr
      symbol	+
      40
      2



Tree(start, [Tree(value, [Tree(sexpr, [Tree(symbol, [Token(SYMBOL, '+')]), 40, '2'])])])

In [58]:
t = LispyPPTransformer()
t.foo = 42


attr = 'int'
try:
    descr = type(t).__dict__[attr]
    try:
        getter = descr.__get__
    except AttributeError:
        res = descr
    else:
        res = getter(t, type(t))
except KeyError:
    res = t.__dict__[attr]
res

int

In [68]:
t.foobar(40) == LispyPPTransformer.foobar(t, 40)

LispyPPTransformer.foobar.__get__(t)

<bound method LispyPPTransformer.foobar of <__main__.LispyPPTransformer object at 0x7f9b000a64a8>>

# Interpretador

O interpretador consiste em uma única função que recebe uma árvore sintática e executa os comandos correspondentes no contexto dado.

In [140]:
def run_ast(ast: Ast, ctx: Ctx):
    """
    Executa árvore sintática no contexto de execução dada.
    """
    
    if isinstance(ast, (float, int)):
        return ast
    elif isinstance(ast, Symbol):
        try:
            return ctx[ast]
        except KeyError:
            raise NameError(f'unknown variable: {ast}')
    
    head, *args = ast
    if head.data == 'define':
        name, value = args
        ctx[name] = result = run_ast(value, ctx)
        return result

    elif head.data == 'if':
        cond, true, false = args
        cond = run_ast(cond, ctx)
        if cond:
            return run_ast(true, ctx)
        else:
            return run_ast(false, ctx)
        
    elif head.data == 'lambda':
        arg_names, body = args
        
        def function(*arg_values):
            local = dict(zip(arg_names, arg_values))
            new_ctx = ChainMap(local, ctx)
            return run_ast(body, new_ctx)
        return function
    
    else:
        func = run_ast(head, ctx)
        args = [run_ast(arg, ctx) for arg in args]
        return func(*args)

In [152]:
#
# Exemplos
#
scheme_factorial = '''
(define fat (lambda (n) 
    (if [n < 2] 
        1
        [n * (fat [n - 1])])))
'''

scheme_factorial = '''
[fat <- (lambda (n) 
    (if [n < 2] 
        1
        [n * (fat [n - 1])]))]
'''

fat = eval_scheme(scheme_factorial)
fat(5)

120